In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import seaborn as sns
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

In [2]:
data = pd.read_csv('telco.csv')

In [3]:
num_col = []
cat_col = []
def col_assign(datas):
    for i in datas.columns:
        if datas[i].dtypes == 'float':
            datas[i] = datas[i].fillna(data[i].mean())
            num_col.append(i)
        else:
            datas[i] = datas[i].fillna(data[i].mode()[0])
            cat_col.append(i)

    return 
col_assign(data)


In [4]:
def standard_scale(datas, col_names):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    datas[col_names] = scaler.fit_transform(datas[col_names])

In [5]:
km = KMeans(n_clusters=3,)

In [6]:
data['Session frequency'] = data.groupby(['Bearer Id'])['Dur. (ms)'].transform('count')

In [7]:
data['Total_Data_Volume(Bytes)'] = data['Total DL (Bytes)'] + data['Total UL (Bytes)']

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 57 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150001 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150001 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        150001 non-null  object 
 10  Avg RTT DL (ms)     

In [9]:
df = pd.DataFrame()
df['Session frequency'] = data['Session frequency']
df["Total_Data"] = data['Total_Data_Volume(Bytes)']
df['Dur. (ms)'] = data['Bearer Id']

In [10]:
standard_scale(df, ['Session frequency', 'Dur. (ms)', 'Total_Data'])

In [11]:
prediction = km.fit_predict(df[['Total_Data', 'Dur. (ms)', 'Session frequency']])
prediction

array([1, 1, 1, ..., 1, 1, 2])

In [12]:
df['cluster'] = prediction 

df1 = df[df.cluster==0]
df2 = df[df.cluster==1]
df3 = df[df.cluster==2]

In [13]:
df['cluster'].value_counts()

0    75566
1    73444
2      991
Name: cluster, dtype: int64

In [14]:
def fix_outlier(df, column):
    df[column] = np.where(df[column]>df[column].quantile(0.95),
                         df[column].mean(),
                         df[column])
    return df[column]
fix_outlier(data,['Avg RTT DL (ms)', 'Avg RTT UL (ms)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)'])


,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,42.000000,5.000000,2.080991e+07,759658.664811,23.000000,44.000000
1,65.000000,5.000000,2.080991e+07,759658.664811,16.000000,26.000000
2,109.795706,17.662883,2.080991e+07,759658.664811,6.000000,9.000000
3,109.795706,17.662883,2.080991e+07,759658.664811,44.000000,44.000000
4,109.795706,17.662883,2.080991e+07,759658.664811,6.000000,9.000000
...,...,...,...,...,...,...
149996,32.000000,0.000000,2.080991e+07,759658.664811,52.000000,65.000000
149997,27.000000,2.000000,2.080991e+07,759658.664811,23.000000,54.000000
149998,43.000000,6.000000,2.080991e+07,759658.664811,43.000000,47.000000
149999,37.000000,5.000000,2.080991e+07,759658.664811,34.000000,37.000000


In [15]:
new_data = pd.DataFrame()
new_data['Avg_RTT'] = data['Avg RTT DL (ms)'] + data['Avg RTT UL (ms)']
new_data['Avg TCP'] = data['TCP DL Retrans. Vol (Bytes)'] + data['TCP UL Retrans. Vol (Bytes)']
new_data['Handset_type'] = data['Handset Type']
new_data['Avg TP'] = data['Avg Bearer TP DL (kbps)']  + data['Avg Bearer TP UL (kbps)']
new_data['MSISDN'] = data['MSISDN/Number']

In [16]:
num_col = []
cat_col = []
def col_assign(df):
    for i in df.columns:
        if df[i].dtypes == 'float':
            df[i] = df[i].fillna(df[i].mean())
            num_col.append(i)
        else:
            df[i] = df[i].fillna(df[i].mode()[0])
            cat_col.append(i)
col_assign(new_data)

In [17]:
scale = StandardScaler()
scaled = ['Avg TCP','Avg_RTT', 'Avg TP' ]
new_data[scaled] = scale.fit_transform(new_data[scaled])

In [18]:
new_data

,Avg_RTT,Avg TCP,Handset_type,Avg TP,MSISDN
0,-0.703698,0.761222,Samsung Galaxy A5 Sm-A520F,-0.627855,3.366496e+10
1,-0.185776,0.761222,Samsung Galaxy J5 (Sm-J530),-0.629366,3.368185e+10
2,1.108095,0.761222,Samsung Galaxy A8 (2018),-0.630997,3.376063e+10
3,1.108095,0.761222,undefined,-0.626586,3.375034e+10
4,1.108095,0.761222,Samsung Sm-G390F,-0.630997,3.369980e+10
...,...,...,...,...,...
149996,-1.041473,0.761222,Apple iPhone 8 Plus (A1897),-0.624834,3.365069e+10
149997,-1.109028,0.761222,Apple iPhone Se (A1723),-0.627251,3.366345e+10
149998,-0.658661,0.761222,Apple iPhone Xs (A2097),-0.626466,3.362189e+10
149999,-0.816290,0.761222,Huawei Fig-Lx1,-0.627614,3.361962e+10


In [19]:
prediction = km.fit_predict(new_data[['Avg_RTT', 'Avg TCP','Avg TP']])
prediction

array([1, 1, 2, ..., 1, 1, 2])

In [20]:
new_data['cluster'] = prediction 

cluster_1 = new_data[new_data.cluster==0]
cluster_2 = new_data[new_data.cluster==1]
cluster_3 = new_data[new_data.cluster==2]

In [21]:
new_data['cluster'].value_counts()

1    58334
0    53930
2    37737
Name: cluster, dtype: int64

In [22]:
cluster_1['Experience_score'] =np.sqrt(((cluster_1['Avg_RTT'] +cluster_1['Avg TCP'] + cluster_1['Avg TP']).pow(2)) - (cluster_1['cluster'].pow(2)))

<ipython-input-22-b5910db76030>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_1['Experience_score'] =np.sqrt(((cluster_1['Avg_RTT'] +cluster_1['Avg TCP'] + cluster_1['Avg TP']).pow(2)) - (cluster_1['cluster'].pow(2)))


In [23]:
df3['Engagement_score'] =np.sqrt(((df3['Total_Data'] +df3['Dur. (ms)'] + df3['Session frequency']).pow(2)) - (df3['cluster'].pow(2)))

<ipython-input-23-43418b0901a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Engagement_score'] =np.sqrt(((df3['Total_Data'] +df3['Dur. (ms)'] + df3['Session frequency']).pow(2)) - (df3['cluster'].pow(2)))


In [24]:
df3['Engagement_score'].value_counts()

12.898210    1
12.457217    1
12.137628    1
11.422840    1
13.247789    1
            ..
10.780354    1
11.341927    1
10.474805    1
10.610531    1
10.659159    1
Name: Engagement_score, Length: 991, dtype: int64

In [25]:
cluster_1['Experience_score'].value_counts()

3.190714    2
0.558300    1
1.927727    1
1.833762    1
0.056785    1
           ..
1.910367    1
0.446203    1
1.211142    1
1.630222    1
1.178758    1
Name: Experience_score, Length: 53929, dtype: int64

In [26]:
df_data = pd.concat([cluster_1, df3], axis=1,join='inner')

In [27]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 616 entries, 463 to 149723
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Avg_RTT            616 non-null    float64
 1   Avg TCP            616 non-null    float64
 2   Handset_type       616 non-null    object 
 3   Avg TP             616 non-null    float64
 4   MSISDN             616 non-null    float64
 5   cluster            616 non-null    int32  
 6   Experience_score   616 non-null    float64
 7   Session frequency  616 non-null    float64
 8   Total_Data         616 non-null    float64
 9   Dur. (ms)          616 non-null    float64
 10  cluster            616 non-null    int32  
 11  Engagement_score   616 non-null    float64
dtypes: float64(9), int32(2), object(1)
memory usage: 57.8+ KB


In [28]:
df_data['Satisfactory_score']= (df_data['Engagement_score']+ df_data['Experience_score'])/2

In [29]:
def minmax_scale(datas, col_names):
    scaler = MinMaxScaler(feature_range=(0, 1))
    datas[col_names] = scaler.fit_transform(datas[col_names])
minmax_scale(df_data, ['Engagement_score', 'Experience_score'])

In [30]:
df_data.drop('cluster', axis=1, inplace=True)

In [31]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 616 entries, 463 to 149723
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Avg_RTT             616 non-null    float64
 1   Avg TCP             616 non-null    float64
 2   Handset_type        616 non-null    object 
 3   Avg TP              616 non-null    float64
 4   MSISDN              616 non-null    float64
 5   Experience_score    616 non-null    float64
 6   Session frequency   616 non-null    float64
 7   Total_Data          616 non-null    float64
 8   Dur. (ms)           616 non-null    float64
 9   Engagement_score    616 non-null    float64
 10  Satisfactory_score  616 non-null    float64
dtypes: float64(10), object(1)
memory usage: 57.8+ KB


In [32]:
df_data['Satisfactory_score'].nlargest(10)

91011     9.021257
112821    8.431575
146672    8.257378
121174    8.206033
137284    8.180133
36568     8.154298
15713     8.044080
13510     8.021849
146599    8.002494
97900     7.981732
Name: Satisfactory_score, dtype: float64

In [33]:
X = df_data.drop(['Satisfactory_score'], axis=1)
y = df_data.Satisfactory_score


In [34]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(492, 10) (124, 10) (492,) (124,)


In [35]:
categorical_features_indices = np.where(X.dtypes == np.object)[0]

<ipython-input-35-636372c33cfd>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(X.dtypes == np.object)[0]


In [36]:
model=CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1)
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_val, y_val))

0:	learn: 0.5659777	test: 0.5521698	best: 0.5521698 (0)	total: 173ms	remaining: 8.49s
1:	learn: 0.5245973	test: 0.5111507	best: 0.5111507 (1)	total: 205ms	remaining: 4.92s
2:	learn: 0.4889131	test: 0.4732455	best: 0.4732455 (2)	total: 231ms	remaining: 3.62s
3:	learn: 0.4598017	test: 0.4416166	best: 0.4416166 (3)	total: 256ms	remaining: 2.95s
4:	learn: 0.4311342	test: 0.4136871	best: 0.4136871 (4)	total: 287ms	remaining: 2.59s
5:	learn: 0.4077650	test: 0.3872715	best: 0.3872715 (5)	total: 323ms	remaining: 2.37s
6:	learn: 0.3834809	test: 0.3606944	best: 0.3606944 (6)	total: 356ms	remaining: 2.18s
7:	learn: 0.3579346	test: 0.3349292	best: 0.3349292 (7)	total: 394ms	remaining: 2.07s
8:	learn: 0.3350416	test: 0.3125404	best: 0.3125404 (8)	total: 432ms	remaining: 1.97s
9:	learn: 0.3170874	test: 0.2957141	best: 0.2957141 (9)	total: 494ms	remaining: 1.98s
10:	learn: 0.2982809	test: 0.2767856	best: 0.2767856 (10)	total: 582ms	remaining: 2.06s
11:	learn: 0.2797641	test: 0.2561221	best: 0.2561221

In [37]:
from sklearn.metrics import r2_score
pred = model.predict(X_val)
r2 = r2_score(y_val, pred)
print(r2)

0.9877984992401372


In [38]:
pred

array([6.76993354, 6.76608061, 5.64556823, 6.41555643, 6.4595867 ,
       6.25857735, 7.09260733, 6.54853337, 6.21299673, 7.16112593,
       7.87536236, 7.13898312, 6.40186155, 5.58615603, 6.78747814,
       5.71146009, 6.04740497, 7.23427993, 7.04855437, 6.033422  ,
       7.95938512, 6.33790446, 5.66590676, 7.09076966, 6.19711633,
       5.54860627, 5.88317836, 7.2002854 , 7.17238072, 6.07498107,
       5.76980413, 6.92599606, 7.2549741 , 6.31778365, 7.09821844,
       6.51561802, 5.90215022, 6.63542257, 6.63835769, 6.50650685,
       6.00073072, 6.33798044, 6.48157496, 7.79456386, 5.78442627,
       5.78509359, 6.01774936, 6.00964123, 6.71834104, 6.1487877 ,
       6.71557763, 6.57418432, 6.41732499, 5.64574424, 5.74650436,
       5.7756183 , 5.98243529, 7.37183223, 7.0083982 , 6.91523689,
       6.21155535, 7.1723951 , 5.6413454 , 6.70497459, 5.68672855,
       5.60465077, 7.1747512 , 6.22657947, 6.81905868, 5.85257167,
       6.29300471, 6.37743225, 6.63136486, 6.58616223, 6.04517

In [39]:
km = KMeans(n_clusters=2)
preds = km.fit_predict(df_data[['Engagement_score', 'Experience_score']])
df_data['cluster']  = preds

In [40]:
satisfied_per_cluster = df_data.groupby('cluster').agg({'Satisfactory_score':["median","mean"]})
experienced_per_cluster = df_data.groupby('cluster').agg({'Experience_score':["median","mean"]})
print(satisfied_per_cluster)
print(experienced_per_cluster)

        Satisfactory_score          
                    median      mean
cluster                             
0                 6.145191  6.180649
1                 6.995635  7.033426
        Experience_score          
                  median      mean
cluster                           
0               0.226447  0.249593
1               0.223827  0.242379


In [41]:
export = pd.DataFrame()
export['User_id']= df_data['MSISDN']
export['Experience_score'] = df_data['Experience_score']
export['Engagement_score'] = df_data['Engagement_score']
export['Satisfactory_score'] = df_data['Satisfactory_score']
minmax_scale(export, ['Satisfactory_score'])

In [49]:
export['Satisfactory_score'].nlargest(10)

91011     1.000000
112821    0.843752
146672    0.797595
121174    0.783990
137284    0.777128
36568     0.770282
15713     0.741078
13510     0.735187
146599    0.730059
97900     0.724557
Name: Satisfactory_score, dtype: float64

In [42]:
export

,User_id,Experience_score,Engagement_score,Satisfactory_score
463,4.188282e+10,0.911730,0.119848,0.532570
478,4.188282e+10,0.066140,0.188901,0.100011
503,4.188282e+10,0.458738,0.479370,0.456058
4176,3.367433e+10,0.113893,0.801114,0.420938
4365,4.188282e+10,0.129511,0.667235,0.365113
...,...,...,...,...
148425,4.188282e+10,0.045073,0.477597,0.227339
148565,4.188282e+10,0.131932,0.440748,0.257452
149206,4.188282e+10,0.177351,0.548512,0.334331
149375,4.188282e+10,0.076832,0.732456,0.367481


In [43]:
import pymysql

In [44]:
connection = pymysql.connect(host='localhost', 
                             user = 'root',
                             password = 'Timileyin52!',
                             db = 'export')

In [45]:
cursor = connection.cursor()

In [46]:
cols = "`,`".join([str(i) for i in export.columns.tolist()])

In [47]:
for i, row in export.iterrows():
    sql = "INSERT INTO `Dataframe` (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    connection.commit()